# Translator Use Case Question 5: Hypertension and Cymbalta

## Understanding the Question

The Translator Use Case Question #5 is:    

> If a patient with disease X is treated off-label with drug Y, what are some potential side effects?

We interpret the Translator Use Case Question to be about **unintended drug-disease interactions**. These occur when a drug has an unintended effect on a person due to the person’s existing medical conditions. 
* The interaction could be beneficial: for example, a person taking [lorazepam](https://www.drugs.com/ppa/lorazepam.html) for nighttime anxiety may also experience drowsiness (a common side effect), which may help them with their insomnia. 
* On the other hand, the interaction could be harmful: for example, a person with peptic ulcers may want to take [ibuprofen](https://www.drugs.com/ppa/ibuprofen.html) for a headache. However, NSAIDS can increase the risk of serious gastrointestinal inflammation, ulceration, bleeding, and perforation, especially for people with a history of GI ulcers or current GI ulcers. Some adverse drug-disease interactions, like the example here, are described in formal drug warnings and contraindications.

We notice that unintended drug-disease interactions occur in more contexts than off-label drug use. They can also occur when the drug is used following its label-indication and in [the context of comorbidity / multimorbidity](https://www.bmj.com/content/350/bmj.h949), when a person is taking a drug to treat one of their diseases and the drug affects a comorbid condition.

**We therefore decided to reframe this question and find potential unintended drug-disease interactions with the following type of question:**
> What symptoms do `Disease` X and `ChemicalSubstance` Y have in common? 

BioThings Explorer (BTE) can answer two classes of queries -- "EXPLAIN" and "PREDICT". This Question fits the EXPLAIN  template of starting with **a specific biomedical entity** (a specific `Disease` X) and finding indirect relationships with **another specific biomedical entity** (a specific `ChemicalSubstance` Y).

## Specific use case: Cymbalta use exacerbating hypertension

We will use **hypertension** as our specific disease of interest. We will use **Cymbalta** as our drug of interest. This use case is an example of comorbidity and unintended drug-disease interactions. Many people have both hypertension and a [disease that can be treated with Cymbalta](https://www.drugs.com/ppa/duloxetine.html) (references: [hypertension and psychiatric disease, including depression and anxiety](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3996437/), [hypertension and fibromyalgia](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4575027/)). 

However, [Cymbalta / duloxetine](https://www.accessdata.fda.gov/drugsatfda_docs/label/2007/021427s009s011s013lbl.pdf) and [other drugs in its class](https://pubmed.ncbi.nlm.nih.gov/30838456/) (serotonin and norepinephrine reuptake inhibitors, SNRIs) can increase blood pressure, which could exacerbate a pre-existing hypertension. This adverse effect is described in [Cymbalta's warnings](https://www.drugs.com/ppa/duloxetine.html), along with advice to use Cymbalta with caution in patients with hypertension, monitor patients' blood pressure periodically while they take Cymbalta, and reduce or discontinue its use if patients experience sustained higher blood pressure from its use. 

We tackle the search for side-effects associated with using Cymbalta and having hypertension using the query:  
* `PhenotypicFeature` hypertension  &rarr; results:`Disease` &larr; `ChemicalSubstance` Cymbalta. 
    * Note that we use the `PhenotypicFeature` entity for hypertension and `Disease` as the intermediate node type to represent disease symptoms and drug effects. With the current APIs available and issues around BioThings type annotation (PhenotypicFeatures vs Diseases), it made more sense to use these types for these nodes.  
* The query will return a graph object with entities as nodes and relationships as edges. We then use edge provenance information to **filter** the results. For each intermediate Disease node, we use the number of unique paths from the input nodes to that node to **score** it. The scores can then be used to sort the results.    

## Step 0: Load BTE modules, notebook functions

In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import modules from biothings_explorer
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## show time that this notebook was executed 
from datetime import datetime

## packages to work with objects 
import re

In [2]:
## functions to add to modules?
def hint_display(query, hint_result):
    """
    show the type, name, number of IDs for all results returned by the query
    
    :param: query: string used in hint query
    :param: hint_result: object returned from hint query, a dictionary of lists of dictionaries
    
    Returns: None
    """
    ## function needs to be rewritten if it's going to give the exact index of each object within its type 
    display = ['type', 'name']  ## replace with the parts of the BioThings object you want to see
    concise_results = []
    for BT_type, result in hint_result.items():
        if result:  ## basically if it's not empty
            for items in result:
                ## number of identifiers per object: number of keys - 4 (name, primary, display, type)
                temp = len(items) - 4
                concise_results.append((items[display[0]], items[display[1]], 
                                         str(temp)))
                    
    print('There are {total} BioThings objects returned for {ht}:'.format(\
                total = len(concise_results), ht = query))
    for display_info in concise_results:
        print('{0}, {1}, num of IDs: {2}'.format(display_info[0], display_info[1], display_info[2]))

In [3]:
def filter_table(df):
    """
    use _source and _method columns to remove rows (paths) from the dataframe
    :param: pandas dataframe containing results from BTE FindConnection module, in table form
    
    Returns: filtered dataframe
    """
    ## note: still needs checking with EXPLAIN queries
    ## key is the string to match to column, value is a list of strings to match to column values
    filter_out = {'_source': ['SEMMED', 'CTD', 'ctd', 'omia']   
#                   '_method': []  ## currently no method stuff I want to filter out
                 }
    ## SEMMED: text mining results wrong for PhenotypicFeature -> Gene
    ## CTD/ctd: results odd for MSUD -> ChemicalSubstance
    ## omia: results wrong or discontinued gene IDs for PhenotypicFeature -> Gene
    
    
    df_temp = df.copy()  ## so the original df isn't modified in-place
    for key,val in filter_out.items():
        ## find columns that match the key string
        columns = [i for i in df_temp.columns if key in i]
        ## iterate through each column
        for col in columns:
            ## iterate through each value to take out, check if string CONTAINS match. 
            ## only keep rows that don't contain the value
            for i in val:
                df_temp = df_temp[~ df_temp[col].str.contains(i, na = False)]
    return df_temp

In [4]:
def scoring_output(df, q_type):
    """
    score results based on whether query was Predict or Explain type, number of 
        intermediate nodes 
    :param: pandas dataframe containing results from BTE FindConnection module
    :param: string describing type of query (Predict or Explain)
    
    May flatten some edges, because score only counts one edge per 
        unique predicate / API / method (ignoring source and pubmed col)
    
    Predict queries: score each output node by counting # of paths
        from input nodes to it. Normalize by dividing by maximum
        possible # of paths
    Explain two-hop (one intermediate) queries: score each intermediate node by 
        counting # of paths (between input and output nodes) that include it. 
        Normalize by dividing by maximum possible # of paths    

    Explain one-hop (direct) queries: no need to score, prints message
    Other Explain queries (many-hops): currently not able to score, prints message     
    
    Returns: pandas series with scores, index is output_name
             or None (one-hop or many-hop Explain query)
    """
    df_temp = df.copy()  ## so no chance to mutate this   
    flag_direct = False  ## one-hop query or not
    ## use df_col to look quicker into columns
    df_col = set(df_temp.columns)
    
    ## ignore source and pubmed col in looking at unique edges 
    columns_drop = [col for col in df_col if (('_source' in col) or ('_pubmed' in col))]
    df_temp.drop(columns = columns_drop, inplace = True)    
    df_temp.drop_duplicates(inplace = True)
    
    ## check if query is one-hop or not
    if "node1_name" not in df_col:    ## name for first intermediate node layer
        flag_direct = True  
    
    if q_type == 'Explain':
        if flag_direct:   # one hop / no intermediates
            print('No valid node scoring for one-hop (direct) Explain queries.')
            return None
        ## if there are many-hops/intermediate layers
        elif "node2_name" in df_col:  ## name for 2nd intermed. node layer
            print('Cannot currently score many-hop Explain queries.')
            return None
        else:   ## two-hop / 1 intermediate layer
            ## count multi-edges to results (the intermediate node1 col)
            scores = df_temp.node1_name.value_counts() 
            ## to find the maximum-possible number of edges, look at non-result cols
            columns_drop = [col for col in df_col if 'node1' in col]
            df_temp.drop(columns = columns_drop, inplace = True)
            ## now look at number of unique combos for input, edge info, output
            df_temp.drop_duplicates(inplace = True)
            max_paths = df_temp.shape[0]            
            ## normalize scores by dividing each by max number of paths
            scores = scores / max_paths
            ## return scores as pandas dataframe
            return scores.to_frame(name = 'score')            

    else:  ## Predict type query
        ## count multi-edges to results (the output col)
        scores = df_temp.output_name.value_counts()
        ## to find the maximum number of multi-edges, look at non-output col
        columns_drop = [col for col in df_temp.columns if 'output' in col]
        df_temp.drop(columns = columns_drop, inplace = True)
        ## now look at number of unique paths possible
        df_temp.drop_duplicates(inplace = True)
        max_paths = df_temp.shape[0]
        ## normalize scores by dividing each by max number of paths
        scores = scores / max_paths
        ## return scores as pandas dataframe
        return scores.to_frame(name = 'score')                 

In [5]:
## record when cell blocks are executed
print('The time that this notebook was executed is...')
print('Local time (PST, West Coast USA): ')
print(datetime.now())
print('UTC time: ')
print(datetime.utcnow())

The time that this notebook was executed is...
Local time (PST, West Coast USA): 
2020-09-14 18:24:25.319790
UTC time: 
2020-09-15 01:24:25.319972


## Step 1: Find representation of "hypertension" in BTE

In this step, BioThings Explorer translates our query string "hypertension"  into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want (the rare disease). 

Generally, the top result returned by the Hint module for your BioThings type of interest will match what you want, but you should confirm that using the identifiers shown. 


> BioThings types correspond to children and descendants of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `Disease` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle"). **However, [only a subset of the Biolink BiologicalEntity children / descendants are currently implemented in BTE](https://smart-api.info/portal/translator/metakg)**. More biomedical object types will be available as more knowledge sources (APIs) are added to the system. **Note that the type `BiologicalEntity` means any BioThings type currently implemented in BTE will be accepted.**

In [6]:
ht = Hint()  ## neater way to call this BTE module

## the human user gives this input
hypert_starting_str = "hypertension"

hypert_hint = ht.query(hypert_starting_str)

hint_display(hypert_starting_str, hypert_hint)

There are 15 BioThings objects returned for hypertension:
Gene, hypertension with brachydactyly, num of IDs: 2
Gene, hypertension, essential, susceptibility to, 8, num of IDs: 2
Gene, Hypertension, essential, susceptibility to, 6, num of IDs: 2
Gene, Renal failure, progressive, with hypertension, num of IDs: 2
Gene, Hypertension, essential, susceptibility to, 2, num of IDs: 2
Disease, essential hypertension, num of IDs: 4
Disease, malignant essential hypertension, num of IDs: 3
Disease, hypertension, pregnancy-induced, num of IDs: 3
Disease, renovascular hypertension (disease), num of IDs: 4
Disease, portal hypertension, noncirrhotic, num of IDs: 3
PhenotypicFeature, Hypertension, num of IDs: 3
PhenotypicFeature, Renovascular hypertension, num of IDs: 3
PhenotypicFeature, Maternal hypertension, num of IDs: 2
PhenotypicFeature, Portal hypertension, num of IDs: 3
PhenotypicFeature, Episodic hypertension, num of IDs: 2


Based on the information above, we'll pick the top `PhenotypicFeature` choice (indexed at 0) for our query. We can look at identifier mappings inside this BioThings object. 

Note that the query didn't work when picking the top `Disease` choice (essential hypertension). 

In [7]:
## the human user makes this choice, gives this input
hypert_choice_type = 'PhenotypicFeature'
hypert_choice_idx = 0

hypert_hint_obj = hypert_hint[hypert_choice_type][hypert_choice_idx]  
hypert_hint_obj

{'UMLS': 'C0020538',
 'HP': 'HP:0000822',
 'MESH': 'D006973',
 'name': 'Hypertension',
 'primary': {'identifier': 'UMLS',
  'cls': 'PhenotypicFeature',
  'value': 'C0020538'},
 'display': 'UMLS(C0020538) HP(HP:0000822) MESH(D006973) name(Hypertension)',
 'type': 'PhenotypicFeature'}

## Step 2: Find representation of "Cymbalta" in BTE

In this step, BioThings Explorer translates our query string "Cymbalta"  into BioThings objects, which contain mappings to many common identifiers. We then pick the BioThings object that best matches what we want (the rare disease). 

Generally, the top result returned by the Hint module for your BioThings type of interest will match what you want, but you should confirm that using the identifiers shown. 


> BioThings types correspond to children and descendants of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `Disease` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle"). **However, [only a subset of the Biolink BiologicalEntity children / descendants are currently implemented in BTE](https://smart-api.info/portal/translator/metakg)**. More biomedical object types will be available as more knowledge sources (APIs) are added to the system. **Note that the type `BiologicalEntity` means any BioThings type currently implemented in BTE will be accepted.**

In [8]:
## the human user gives this input
cymbalta_starting_str = "Cymbalta"

cymbalta_hint = ht.query(cymbalta_starting_str)
## the object is a dictionary with keys = set strings, values = list of dictionaries. 
hint_display(cymbalta_starting_str, cymbalta_hint)

There are 4 BioThings objects returned for Cymbalta:
ChemicalSubstance, (S)-duloxetine hydrochloride, num of IDs: 5
ChemicalSubstance, Cymbalta, num of IDs: 2
ChemicalSubstance, DULOXETINE, num of IDs: 12
ChemicalSubstance, DULOXETINE HYDROCHLORIDE, num of IDs: 11


All of these `ChemicalSubstance` entries seem to be the right object. We'll pick the `ChemicalSubstance` choice with the most identifiers (indexed at 2) for our query. We can look at identifier mappings inside this BioThings object. 

## Step 3: hypertension &rarr; Disease &larr; Cymbalta

In [9]:
## the human user makes this choice, gives this input
cymbalta_choice_type = 'ChemicalSubstance'
cymbalta_choice_idx = 2

cymbalta_hint_obj = cymbalta_hint[cymbalta_choice_type][cymbalta_choice_idx]  
cymbalta_hint_obj
## these inner dictionaries are keys = id type, 
##       values = curie, normal string, or dictionary (for the key 'primary')

{'CHEMBL.COMPOUND': 'CHEMBL1175',
 'DRUGBANK': 'DB00476',
 'PUBCHEM': 60835,
 'CHEBI': 'CHEBI:36795',
 'UMLS': 'C1505021',
 'MESH': 'D000068736',
 'UNII': '9044SC542W',
 'INCHIKEY': 'ZEUITGRIYCTCEM-KRWDZBQOSA-N',
 'INCHI': 'InChI=1S/C18H19NOS/c1-19-12-11-17(18-10-5-13-21-18)20-16-9-4-7-14-6-2-3-8-15(14)16/h2-10,13,17,19H,11-12H2,1H3/t17-/m0/s1',
 'name': 'DULOXETINE',
 'CAS': '116539-59-4',
 'IUPAC': 'methyl-[(3S)-3-(1-naphthoxy)-3-(2-thienyl)propyl]amine',
 'formula': 'C18H19NOS',
 'primary': {'identifier': 'CHEBI',
  'cls': 'ChemicalSubstance',
  'value': 'CHEBI:36795'},
 'display': 'CHEBI(CHEBI:36795) CHEMBL.COMPOUND(CHEMBL1175) DRUGBANK(DB00476) PUBCHEM(60835) MESH(D000068736) UNII(9044SC542W) UMLS(C1505021) name(DULOXETINE) CAS(116539-59-4) IUPAC(methyl-[(3S)-3-(1-naphthoxy)-3-(2-thienyl)propyl]amine) formula(C18H19NOS)',
 'type': 'ChemicalSubstance'}

In this section, we dynamically generate a knowledge graph with paths connecting the PhenotypicFeature hypertension and the drug Cymbalta to Diseases (symptoms / side effects).  

BTE performs the **query path planning** and **query path execution** by deconstructing the query into individual API calls, executing those API calls, and then assembling the results.

The code block below takes ~5 seconds to run. 

In [10]:
## the human user gives this input
q1_intermediate = 'Disease'

q1 = FindConnection(input_obj = hypert_hint_obj,\
                    output_obj = cymbalta_hint_obj, \
                    intermediate_nodes = q1_intermediate)
q1.connect(verbose = False)

API 2.2 pharos failed
API 2.1 pharos failed


In [11]:
# q1_r_graph = q1.fc.G  ## for changing the graph object to reflect the table
q1_r_paths_table = q1.display_table_view()

q1_type = re.findall("dispatcher.([a-zA-Z]+)'", str(type(q1.fc)))
q1_type = "".join(q1_type)  ## convert to string

q1 = None  ## clear memory

We can see the number of Diseases that were linked to both hypertension and to Cymbalta and the total number of paths from the hypertension node to the Cymbalta node.

In [12]:
## show number of unique intermediate nodes
print("There are {0} unique {1}s linked to both {2} and {3}.".format( \
    q1_r_paths_table.node1_name.nunique(), q1_intermediate, hypert_starting_str, cymbalta_starting_str))

## show number of paths from hypertension to cymbalta
print("There are {0} unique paths between {1} and {2}.".format( \
    q1_r_paths_table.shape[0], hypert_starting_str, cymbalta_starting_str))

There are 2 unique Diseases linked to both hypertension and Cymbalta.
There are 3 unique paths between hypertension and Cymbalta.


### Filtering and scoring

Filtering involves using edge provenance, like the source this relationship came from and the method used to make this association, to filter out edges (removing nodes in the process). 

In [13]:
q1_r_paths_table = filter_table(q1_r_paths_table)

## show number of paths from MSUD to genes
print("There are {0} unique paths.".format( \
    q1_r_paths_table.shape[0]))

There are 2 unique paths.


The scoring process for two-hop Explain queries (the type of query we're using now, has one intermediate step): 

1. To score individual intermediate nodes (Diseases), we first take a copy of the knowledge graph (KG) and remove some multi-edges. 
    * Each edge has predicate, API, method, source, and pubmed information. For scoring purposes, we will ignore pubmed and source information because APIs handle this information differently (returning multiple edges or single edges). 
2. We then count the number of edges to each intermediate node (from hypertension and Cymbalta nodes).        
3. Finally, we "normalize" the score by dividing those counts by maximum-possible number of edges to an intermediate node (from hypertension and Cymbalta nodes).

We can then see the top-scored intermediate nodes. A score of closer to 1 means that the many relationships link this node to hypertension and Cymbalta. A score closer to 0 means that only a few relationships link this node to hypertension and Cymbalta. 

In [14]:
## create scoring table for results
q1_scoring = scoring_output(q1_r_paths_table, q1_type)

q1_scoring.head(10)

,score
NEUROLEPTIC MALIGNANT SYNDROME,0.5
"BLOOD PRESSURE, HIGH",0.5


Different knowledge sources (APIs) were called in different parts of the query. 

In the first part of the query (hypertension &rarr; Disease), the following APIs returned results and the following predicates (semantic relationships) were found.

In [15]:
## show that the APIs use different predicates
q1_r_paths_table[['pred1_api', 'pred1']].drop_duplicates().sort_values(by = ['pred1_api', 'pred1'])

,pred1_api,pred1
1,BioLink API,disease_has_basis_in_feature
2,BioLink API,has_phenotype


In the second part of the query (Cymbalta &rarr; Disease), the following APIs returned results and the following predicates (semantic relationships) were found.

In [16]:
## show that the APIs use different predicates
q1_r_paths_table[['pred2_api', 'pred2']].drop_duplicates().sort_values(by = ['pred2_api', 'pred2'])

,pred2_api,pred2
1,MyChem.info API,contraindication


[TO DO: Actual code depends on graph object (ReasonerStd object or Networkx MultiDiGraph) used.]

We then add the scores and score provenance information to the graph object that will be returned to the ARS. These stored as node attributes on the Gene nodes of the "answer graph". 

In [17]:
# nx.set_node_attributes(q1_r_graph, values = q1_scoring.to_dict(), \
#                        name = 'score')
# nx.set_node_attributes(q1_r_graph, values = '(0-1]', name = 'score_range')
# nx.set_node_attributes(q1_r_graph, values = 'normalized path count', name = 'score_method')
# ## what is a good score, larger or smaller? 
# nx.set_node_attributes(q1_r_graph, values = 'larger', name = 'score_better_direction')  

# ## example of Gene node object with score-related attributes
# q1_r_graph.nodes()['NDUFS1']

## Evaluate results

In [17]:
q1_r_paths_table

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,pred1_method,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,pred2_method,output_type,output_name,output_id
1,HIGH BLOOD PRESSURE,PhenotypicFeature,disease_has_basis_in_feature,None,BioLink API,None,None,Disease,"BLOOD PRESSURE, HIGH",MONDO:MONDO:0005044,contraindication,drugcentral,MyChem.info API,None,None,Disease,(3S)-N-METHYL-3-(1-NAPHTHYLOXY)-3-(2-THIENYL)P...,name:(3S)-N-METHYL-3-(1-NAPHTHYLOXY)-3-(2-THIE...
2,HIGH BLOOD PRESSURE,PhenotypicFeature,has_phenotype,hpoa,BioLink API,None,None,Disease,NEUROLEPTIC MALIGNANT SYNDROME,MONDO:MONDO:0019790,contraindication,drugcentral,MyChem.info API,None,None,Disease,(3S)-N-METHYL-3-(1-NAPHTHYLOXY)-3-(2-THIENYL)P...,name:(3S)-N-METHYL-3-(1-NAPHTHYLOXY)-3-(2-THIE...


**BTE's results*** include two **potential adverse side effects** of using Cymbalta that are related to hypertension. 
1. **BTE successfully identifies [high blood pressure (and therefore hypertension) as an adverse side effect of Cymbalta (using the contraindication predicate)](https://www.accessdata.fda.gov/drugsatfda_docs/label/2007/021427s009s011s013lbl.pdf).** 
2. Surprisingly, BTE identifies another potential adverse side effect: [neuroleptic malignant syndrome (NMS)](https://www.ninds.nih.gov/Disorders/All-Disorders/Neuroleptic-Malignant-Syndrome-Information-Page). NMS symptoms can include unstable blood pressure (this is the likely connection to high blood pressure). At first glance, it is surprising to link this syndrome with Cymbalta because this syndrome is rare and normally associated with neuroleptic or antipsychotic drugs (Cymbalta is neither of those). However, a search revealed that this association may be valid.
    * [Serotonergic agents like Cymbalta may enhance adverse effects and the dopamine blockade of antipsychotic agents, so a person taking both classes of drugs may be at increased risk for neuroleptic malignant syndrome ](https://www.drugs.com/ppa/duloxetine.html)
    * Related to the point above, [a case report linked sudden duloxetine *withdrawal* and antipsychotic medication treatment with neuroleptic malignant syndrome](https://n.neurology.org/content/88/16_Supplement/P2.219).
    * [A few case reports have linked venlafaxine use (another drug in the SNRI class, like duloxetine) with neuroleptic malignant syndrome](https://pubmed.ncbi.nlm.nih.gov/?term=neuroleptic+malignant+syndrome+venlafaxine)
    
*Note that the actual BTE results (diseases identified and their scores) vary because BTE dynamically generates its knowledge graph from current API results. 